<a href="https://colab.research.google.com/github/lakshmi-devagiri/Linear_Regression/blob/main/LASSO_%26_RIDGE(OVERFITTING).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
from google.colab import files
uploaded = files.upload()



Saving Melbourne_housing_FULL.csv to Melbourne_housing_FULL.csv


In [3]:
df=pd.read_csv('Melbourne_housing_FULL.csv')
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [4]:
#To see unique values in the dataset
df.nunique()

,0
Suburb,351
Address,34009
Rooms,12
Type,3
Price,2871
Method,9
SellerG,388
Date,78
Distance,215
Postcode,211


In [6]:
df.shape

(34857, 21)

Here, there are 34857 rows and 21 columns

In [7]:
# Here, I don't need all the columns so, I am taking only few columns which are needed for analysis
columns=['Suburb','Rooms','Type','Method','Regionname','Propertycount','SellerG','Distance','Bedroom2','Bathroom','Car','Landsize','BuildingArea','Price']
df=df[columns]
df.head()

,Suburb,Rooms,Type,Method,Regionname,Propertycount,SellerG,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price
0,Abbotsford,2,h,SS,Northern Metropolitan,4019.0,Jellis,2.5,2.0,1.0,1.0,126.0,NaN,NaN
1,Abbotsford,2,h,S,Northern Metropolitan,4019.0,Biggin,2.5,2.0,1.0,1.0,202.0,NaN,1480000.0
2,Abbotsford,2,h,S,Northern Metropolitan,4019.0,Biggin,2.5,2.0,1.0,0.0,156.0,79.0,1035000.0
3,Abbotsford,3,u,VB,Northern Metropolitan,4019.0,Rounds,2.5,3.0,2.0,1.0,0.0,NaN,NaN
4,Abbotsford,3,h,SP,Northern Metropolitan,4019.0,Biggin,2.5,3.0,2.0,0.0,134.0,150.0,1465000.0


In [9]:
df.shape

(34857, 14)

In [13]:
df.isna()

,Suburb,Rooms,Type,Method,Regionname,Propertycount,SellerG,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price
0,False,False,False,False,False,False,False,False,False,False,False,False,True,True
1,False,False,False,False,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,True,True
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,False,False,False,False,False,False,False,False,False,False,False,False,True,False
34853,False,False,False,False,False,False,False,False,False,False,False,False,False,False
34854,False,False,False,False,False,False,False,False,False,False,False,False,False,False
34855,False,False,False,False,False,False,False,False,True,True,True,True,True,False


In [14]:
df.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Method,0
Regionname,3
Propertycount,3
SellerG,0
Distance,1
Bedroom2,8217
Bathroom,8226


In [16]:
# Here, I am going to replace some of the columns with zero
columns_to_zero=['Propertycount','Distance','Bedroom2','Bathroom','Car']
df[columns_to_zero]=df[columns_to_zero].fillna(0)
df.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Method,0
Regionname,3
Propertycount,0
SellerG,0
Distance,0
Bedroom2,0
Bathroom,0


In [18]:
#Here, I am replacing some of the columns missing value with mean
columns_to_mean=['Landsize','BuildingArea']
df[columns_to_mean]=df[columns_to_mean].fillna(df[columns_to_mean].mean())
df.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Method,0
Regionname,3
Propertycount,0
SellerG,0
Distance,0
Bedroom2,0
Bathroom,0


In [22]:
# Here, I am dropping few colums
#Inplace =True modifies the DataFrame directly without Creating new copy
#This means the original DataFrame df is updated with the dropped rows containing  missing values in the specified columns (Regionname, Price).
df.dropna(inplace=True)
df.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Method,0
Regionname,0
Propertycount,0
SellerG,0
Distance,0
Bedroom2,0
Bathroom,0


Finally, we can see a dataset without having null values:)

Now, we are going to perform One-Hot-encoding

In [26]:
dataset=pd.get_dummies(df,drop_first=True).astype(int)
dataset.head()

,Rooms,Propertycount,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price,Suburb_Aberfeldie,...,SellerG_hockingstuart/Harcourts,SellerG_hockingstuart/Marshall,SellerG_hockingstuart/Sweeney,SellerG_hockingstuart/Village,SellerG_hockingstuart/hockingstuart,SellerG_iHomes,SellerG_iOne,SellerG_iProperty,SellerG_iSell,SellerG_iTRAK
1,2,4019,2,2,1,1,202,160,1480000,0,...,0,0,0,0,0,0,0,0,0,0
2,2,4019,2,2,1,0,156,79,1035000,0,...,0,0,0,0,0,0,0,0,0,0
4,3,4019,2,3,2,0,134,150,1465000,0,...,0,0,0,0,0,0,0,0,0,0
5,3,4019,2,3,2,1,94,160,850000,0,...,0,0,0,0,0,0,0,0,0,0
6,4,4019,2,3,1,2,120,142,1600000,0,...,0,0,0,0,0,0,0,0,0,0


Here, I am dropping the price because i am taking Price
as independent variable

In [40]:
X=dataset.drop('Price',axis=1)
y=dataset['Price']

In [41]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=0.3,random_state=2)

In [42]:
#Here, I am using linear regression
from sklearn.linear_model import LinearRegression
reg=LinearRegression().fit(train_X,train_y)

In [43]:
reg_score=reg.score(train_X,train_y)
reg_score

0.6825862950046448

In [44]:
reg_score=reg.score(test_X,test_y)
reg_score

0.13411895701661847

Here, My test data has less accuracy than train data because Overfitting the model. To Regularize this we are using LASSO Regression(L1 Regularization) Here I am using that model.

In [45]:
# Here, yoou can take whatever alpha Value you want
from sklearn import linear_model
lasso_reg=linear_model.Lasso(alpha=50,max_iter=100,tol=0.1)
lasso_reg.fit(train_X,train_y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.339e+15, tolerance: 7.963e+14
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=50, max_iter=100, tol=0.1)

# according to the equation
L1 regularization= MSE(Loss function) = 1/n * Σ(yᵢ - ŷᵢ)² + λ * Σ|βⱼ|

In [49]:
Lasso_score=lasso_reg.score(train_X,train_y)
Lasso_score

0.6763727813592064

In [50]:
Lasso_score=lasso_reg.score(test_X,test_y)
Lasso_score

0.6619631945015897

Finally, compare to 13% by using Lasso we have 66% which is a big difference by using lasso regression we acheived.

In [52]:
from sklearn.linear_model import Ridge
ridge_reg=Ridge(alpha=40,max_iter=90,tol=0.1)
ridge_reg.fit(train_X,train_y)

Ridge(alpha=40, max_iter=90, tol=0.1)

In [55]:
Ridge_score=ridge_reg.score(train_X,train_y)
print("Ridge Score:", Ridge_score)

Ridge Score: 0.6598132775307082


In [56]:
Ridge_score=ridge_reg.score(test_X,test_y)
print("Ridge Score:", Ridge_score)

Ridge Score: 0.6650454121906196
